In [ ]:
import pandas as pd
import re
df = pd.read_csv('../input/netflix-shows/netflix_titles.csv')
df = df[['release_year','type','title']]
df.dropna(axis='index',how ='any',inplace=True)
df.reset_index(drop = True,inplace=True)
df['type'] = df['type'].apply(lambda x: re.sub('TV Show','TV Series',x))
df.head(5)

In [ ]:
pip install beautifulsoup4

# IMDB ratings scraper

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import urllib.request, urllib.parse
def rating(x):
    title = x.title
    year  = x.release_year
    category= x.type
    year  ='('+str(year)+')'
    title = str(title).replace('\u200b','').lower() #Some titles had zero_width_space character in the name

    #getting url  

    encoding = 'utf-8' #standard internet encoding
    find_url = 'https://www.imdb.com/find' #base search URL for IMDB
    values = {'q':title} # What we are searching for
    query = urllib.parse.urlencode(values).encode(encoding) #parsing the query and encoding correctly for web to understand it
    req = urllib.request.Request(find_url,query) #request: the "link" you can paste in the search bar
    resp = urllib.request.urlopen(req) #sending the request and recording the response
    
    #Matching movies from search results 
    
    soup = bs(resp,'html')
    soup = soup.find(class_='findList')
    
    if soup==None:              #Return can't find if no matches found
        return('can\'t find')

    for small in soup('small'): #remove small tag
        small.decompose()
        
    box1=soup.find_all(class_='result_text')
    title = re.sub('[^a-z0-9]','',title)
    flag  = False
    
    for i in range(len(box1)):  #Match: Title,Year of release, Type(movie,TV Series)
            if category=='Movie':
                if bool(re.search(title, re.sub('[^a-z0-9]','',box1[i].text.lower()))) and bool(re.search(year, box1[i].text)) and not bool(re.search('(TV Series)|(TV Episode)', box1[i].text)):
                    flag = True
                    break
            else:
                if bool(re.search(title, re.sub('[^a-z0-9]','',box1[i].text.lower())))  and bool(re.search('TV Series', box1[i].text)):
                    flag = True
                    break
    
    #Getting IMDB rating using matched movie title tag
    #Returing the rating, 0 if not found
    
    if flag:                                              #if a match is found
        box =box1[i]                                                                
        source2 = requests.get('https://www.imdb.com'+box.a['href']).text
        soup2=bs(source2,'lxml')
        box2=soup2.find(class_='ratingValue')
        if box2 == None:                                  #this means the matched title has not been rated yet
            return ('not rated')
        return re.search('.+(?=/)',box2.text).group(0)
    else :                                                #if no match found
        return('can\'t find')

In [ ]:
ratings=pd.DataFrame(df.loc[0:50,['title','type','release_year']]) #Checking the ratings of first 50 movies
ratings['ratings']=ratings.apply(rating,axis=1)
ratings

## Known Problems
#### 1. Some movies and TV-shows don't appear on IMDB
#### 2. Netflix and IMDB release dates are not same for some movies and TV-shows
#### 3. Titles of few movies and TV-shows are not exactly similar between Netflix and IMDB
#### 4. This is a bit slow to run
### I am new to this so would appriciate any ideas for improving this.